# Keras Exploration

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

Using TensorFlow backend.


## Data Preprocessing
Pada tahap ini, dilakukan prepocessing terhadap data weather. Data mentah weather terdiri atas 4 atribut, yaitu
- outlook, data nominal dengan range *sunny*, *overcast*, dan *rainy*
- temperature, data bertipe numerik
- humidity, data bertipe numerik
- windy, data bertipe nominal dengan range True atau False

Target pembelajaran adalah label dengan range 'yes' (bermain tenis) atau 'no' (tidak bermain tenis).

Untuk data masukan bertipe nominal, akan digunakan strategi *one hot encoding* sebagai *preprocessing* data. Data bertipe nominal akan dibiarkan (*passthrough*)

Dari dataset yang tersedia, 90% data akan digunakan sebagai data latih dan sisanya sebagai data test. Digunakan skema *hold-out* untuk melakukan evaluasi terhadap model yang dibangun. 

In [2]:
# Loading data
data, meta = arff.loadarff('weather.arff')
data_x = [list(d)[:4] for d in data]
data_y = [d[4] for d in data]

# encoding data
le = LabelEncoder()
data_y = le.fit_transform(data_y)

ct = ColumnTransformer([('ohe', OneHotEncoder(), [0, 3]),], remainder= 'passthrough')
data_x = ct.fit_transform(data_x)

# splitting data into train and test data
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size= 0.1)

## Pembangunan Arsitektur Neural Network

Digunakan neural network dengan satu *hidden layer* dengan banyak neuron 50 buah. Output layer adalah layer dengan satu neuron dan menggunakan fungsi aktivasi sigmoid (supaya output berada pada range (0,1)). 
*Biases* dan *weights* diinisialisasi dengan metode *xavier uniform initializer*. 

In [3]:
dense_size = [32, 8, 1]

model = Sequential()
for i, size in enumerate(dense_size):
    if (i == 0):
        model.add(Dense(size, input_dim= train_x.shape[1], activation= 'sigmoid'))
    elif (i != len(dense_size) -1 ):
        model.add(Dense(size, activation= 'sigmoid', kernel_initializer= 'glorot_uniform', bias_initializer= 'glorot_uniform'))
    else:
        model.add(Dense(size, activation= 'sigmoid', kernel_initializer= 'glorot_uniform', bias_initializer= 'glorot_uniform'))

## Pelatihan Neural Network
Neural network dilatih dengan data latih yang tersedia. Pelatihan dilakukan selama 32 epoch dengan batch size sebesar 2. Fungsi loss yang digunakan adalah *binary crossentropy loss*. Selama pelatihan berlangsung, loss dan accuracy dari model akan dipantau.

In [4]:
sgd = SGD(lr= 0.01, momentum= 1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model.summary()
model.fit(train_x, train_y, epochs= 32, batch_size= 1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 529
Trainable params: 529
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
12/12 [==============================] - 0s 24ms/step - loss: 1.0215 - acc: 0.4167
Epoch 2/32
12/12 [==============================] - 0s 3ms/step - loss: 0.9392 - acc: 0.4167
Epoch 3/32
12/12 [==============================] - 0s 3ms/step - loss: 0.8926 - acc: 0.4167
Epoch 4/32
12/12 [==============================] - 0s 3ms/step - loss: 0.8536 - acc: 0.4167
Epoch 5/32
12/12 [=================

In [5]:
scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

2/2 [==============================] - 0s 20ms/step

acc: 100.00%
